# About Trafg

Trafg takes a packet_count and generates:                           Volume     

Training phase:                                                         
    packet_count/2 legitimate traffic                               1/3
Detection phase:
    packet_count/4 legitimate traffic                               1/6
    packet_count/2 legitimate + malicious traffic:                  1/3
        malicious:         attack_proportion  * packet_count/2      
        legitimate:     (1-attack_proportion) * packet_count/2  
    packet_count/4 legitimate traffic                               1/6

Timestamps in the synthetic workload ddos<proportion%>.txt files:

Line 1: Beginning of the attack.
Line 2: End of the attack. 

# Reverse-Engineering Synthetic Workloads

Libpcap File Format https://wiki.wireshark.org/Development/LibpcapFileFormat
Global header: 24 bytes
Record header: 16 bytes

Analyzing a trafg-generated workload.

    File: datasets/aclapolli/ddos5.pcap

    Pcap record size:           68      bytes
        Record header:          16      
        Capture data:           52
            Ethernet:               14
            IPv4:                   20
            Custom payload:         17
                Timestamp               16 [Offset: 0x22-0x31]
                Attack flag             01 [Offset: 0x32]
            Ethernet:               01

    Total size:     13.369.344.024 bytes
    Global header:              24 bytes  
    Record area:    13.369.344.000 bytes
    # of records:      196.608.000 packets

    Capture Start:	1459947780000000	Wednesday, 06-Apr-16 13:03:00.000000 UTC
    Attack Start:	1459947871852575	Wednesday, 06-Apr-16 13:04:31.852575 UTC
    Attack End:		1459947931138652	Wednesday, 06-Apr-16 13:05:31.138652 UTC
    Capture End:	1459947962808079	Wednesday, 06-Apr-16 13:06:02.808079 UTC

    First Packet:

        "1",			"2016-04-06 13:03:00.000000","161.69.48.219","161.69.45.5","IPv4","52","Unknown (253)"

    Last Packet

        "196608000",	"2016-04-06 13:06:02.808079","5.255.217.159","66.56.90.148","IPv4","52","Unknown (253)"

    Legitimate Traces: 
        
        File                                                Packets
        equinix-chicago.dirA.20160406-130300.UTC.anon.pcap  32386894
        equinix-chicago.dirA.20160406-130400.UTC.anon.pcap  30325344
        equinix-chicago.dirA.20160406-130500.UTC.anon.pcap  32133986
        equinix-chicago.dirA.20160406-130600.UTC.anon.pcap  29793140
        equinix-chicago.dirA.20160406-130700.UTC.anon.pcap  28041980
        equinix-chicago.dirA.20160406-130800.UTC.anon.pcap  31989418
        equinix-chicago.dirA.20160406-130900.UTC.anon.pcap  30598765

    Attack Traces:

        File                                                Packets
        ddostrace.20070804_141436.pcap                      ~25M

    Trafg Parameters:

        Since trafg outputs 1.5 * packet_count = 196.608.000 packets, 
        we know that packet_count = 196.608.000 / 1.5 = 131.072.000 = 2**7 * 2**10 * 1000 .
    
Splitting the workload in 24 volumes with 8.192.000 packets each. 

    Phase       Packet Count    Start           End                Volume  Offset
    Total        196.608.000                                       24/24   00..23        
    Training:     65.536.000    13:03:00.000000 13:04:00.087231     8/24   00..07
    Detection 1:  32.768.000    13:04:00.087231 13:04:31.852544     4/24   08..11
    Detection 2:  65.536.000    13:04:31.852545 13:05:31.138658     8/24   12..19
    Detection 3:  32.768.000    13:05:31.138659 13:06:02.808079     4/24   20..23

    Files: in datasets/aclapolli/ddos5_split/*.pcap

Window counts using m=2^18 packets per window.

    Phase       Window Count    Start           End                Volume  Offset
    Total                750                                       24/24   00..23        
    Training:            250    13:03:00.000000 13:04:00.087231     8/24   00..07
    Detection 1:         125    13:04:00.087231 13:04:31.852544     4/24   08..11
    Detection 2:         250    13:04:31.852545 13:05:31.138658     8/24   12..19
    Detection 3:         125    13:05:31.138659 13:06:02.808079     4/24   20..23


# Starting Several Wireshark Instances

    for i in {1..5..2}; do wireshark -i veth$i -k & done

# Starting the Switch 

Using the 'pure' switch: 

    ./veth.sh setup 8
    ./run.sh 

Using the 'mininet' environment: 

    ./mininet.sh 
    ./control.sh
    
    

# Fixed-Point Representations

    Alpha       24.8   (/256) 
    K           29.3   (/8)
    Entropy	    28.4   (/16)
    Ewmmd	    14.18  (/2^18)

# Custom Hash Functions 

In aclapolli-bmv2/targets/simple_switch/simple_switch.cpp:60 

    struct h1 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(155878125)*data + 144393013) % 179424691) % 976;
        }
        return hash;
    }
    };

    struct h2 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(113292259)*data + 68483982) % 179424691) % 976;
        }
        return hash;
    }
    };

    struct h3 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(35316782)*data + 126831809) % 179424691) % 976;
        }
        return hash;
    }
    };

    struct h4 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(159559853)*data + 107589136) % 179424691) % 976;
        }
        return hash;
    }
    };
    
    struct g1 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(81840042)*data + 13527551) % 179424691) % 2;
        }
        return hash;
    }
    };

    struct g2 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(19537968)*data + 97145494) % 179424691) % 2;
        }
        return hash;
    }
    };

    struct g3 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(50123735)*data + 142091935) % 179424691) % 2;
        }
        return hash;
    }
    };

    struct g4 {
    uint32_t operator()(const char *buf, size_t s) const {
        uint32_t hash = 0;
        if (s == 4) {
        uint32_t data = ntohl(*reinterpret_cast<const uint32_t*>(buf));
        hash = ((static_cast<uint64_t>(133965917)*data + 4784577) % 179424691) % 2;
        }
        return hash;
    }
    };

# Experimental Environment

PxS Network

On Lab Player

3: enp0s9: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:4e:08:6a brd ff:ff:ff:ff:ff:ff
    inet 10.11.0.1/24 brd 10.11.0.255 scope global enp0s9
       valid_lft forever preferred_lft forever
    inet6 fc80:b::1/64 scope global 
       valid_lft forever preferred_lft forever
    inet6 fe80::4be6:931:7d9e:db4f/64 scope link 
       valid_lft forever preferred_lft forever

On Lab P4

4: enp0s9: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:16:21:b9 brd ff:ff:ff:ff:ff:ff
    inet 10.11.0.8/24 brd 10.11.0.255 scope global enp0s9
       valid_lft forever preferred_lft forever
    inet6 fc80:b::8/64 scope global 
       valid_lft forever preferred_lft forever
    inet6 fe80::2fd8:76e8:530f:2582/64 scope link 
       valid_lft forever preferred_lft forever



In [1]:
!putty -load "VM Lab P4"

In [2]:
!plink -load "VM Lab P4" -batch ls

^C
